Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [1]:
NAME = "Anastasiia Filippova"
COLLABORATORS = ""

---

# I. $LU$ - разложение квадратной матрицы



Рассмотрим наивную реализацию LU - разложения.

Заметим, что мы используем массивы numpy для представления матриц. [Не используйте 'np.matrix'].


In [2]:
import numpy as np

def diy_lu(a):
    """Создает LU - разложение матрицы `a`.
    
    Наивное LU - разложение: работает столбец за столбцом, накапливает элементарные треугольные матрицы.
    Без выбора главного элемента.
    """
    N = a.shape[0]
    
    u = a.copy()
    L = np.eye(N)
    for j in range(N-1):
        lam = np.eye(N)
        gamma = u[j+1:, j] / u[j, j]
        lam[j+1:, j] = -gamma
        u = lam @ u

        lam[j+1:, j] = gamma
        L = L @ lam
    return L, u

In [3]:
# Теперь сгенерируем матрицу полного ранга и протестируем наивное разложение.
import numpy as np

N = 6
a = np.zeros((N, N), dtype=float)
for i in range(N):
    for j in range(N):
        a[i, j] = 3. / (0.6*i*j + 1)

L,U = diy_lu(a)

np.linalg.matrix_rank(a)

6

In [4]:
# Настройка вывода чисел с плавающей точкой для большей ясности
np.set_printoptions(precision=3)

In [5]:
L, u = diy_lu(a)
print(L, "\n")
print(u, "\n")

# Быстрый тест на адекватность: L @ U должна быть равна изначальной матрице с точностью до ошибок округления.
print(a-L@u)

[[1.    0.    0.    0.    0.    0.   ]
 [1.    1.    0.    0.    0.    0.   ]
 [1.    1.455 1.    0.    0.    0.   ]
 [1.    1.714 1.742 1.    0.    0.   ]
 [1.    1.882 2.276 2.039 1.    0.   ]
 [1.    2.    2.671 2.944 2.354 1.   ]] 

[[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [ 0.000e+00 -1.125e+00 -1.636e+00 -1.929e+00 -2.118e+00 -2.250e+00]
 [ 0.000e+00  0.000e+00  2.625e-01  4.574e-01  5.975e-01  7.013e-01]
 [ 0.000e+00  2.220e-16  0.000e+00 -2.197e-02 -4.480e-02 -6.469e-02]
 [ 0.000e+00 -4.528e-16  0.000e+00  6.939e-18  8.080e-04  1.902e-03]
 [ 0.000e+00  4.123e-16  0.000e+00 -1.634e-17  0.000e+00 -1.585e-05]] 

[[ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00 -2.220e-16  1.110e-16  1.665e-16]
 [ 0.000e+00  0.000e+00 -2.220e-16  5.551e-17  1.665e-16  1.665e-16]
 [ 0.000e+00  0.000e+00  1.110e-16  1.665e-16 -1.665e-16 -5.551e-17]
 

# II. Необходимость выбора главного элемента

Давайте немного подправим матрицу, изменив в ней один элемент:

In [6]:
a1 = a.copy()
a1[1, 1] = 3
print(a1)

[[3.    3.    3.    3.    3.    3.   ]
 [3.    3.    1.364 1.071 0.882 0.75 ]
 [3.    1.364 0.882 0.652 0.517 0.429]
 [3.    1.071 0.652 0.469 0.366 0.3  ]
 [3.    0.882 0.517 0.366 0.283 0.231]
 [3.    0.75  0.429 0.3   0.231 0.188]]


Результирующая матрица имеет полный ранг, но наивное LU-разложение не работает.

In [7]:
np.linalg.matrix_rank(a1)

6

In [8]:
l, u = diy_lu(a1)

print(l, u, sep='\n')

[[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]]
[[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]]


/home/kotya/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in true_divide
  from ipykernel import kernelapp as app
/home/kotya/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in matmul
/home/kotya/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in matmul
/home/kotya/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app


### Тест II.1

Для того, чтобы наивное LU - разложение работало необходимо чтобы все лидирующие миноры матрицы были отличны от нуля. Проверьте, выполнено ли это требование для двух матриц a и a1.

In [9]:
def minor(a):
    ''' Check if all leading minors are non-zero.
    
    Parameters
    ----------
    a : np.array
        2D array representing the square matrix
    
    Returns
    -------
    answer : bool
        True if all leading minors are non-zero
    '''
    M = a.shape[0]
    for j in range (M+1):
        sub_a = a[0:j, 0:j]
        print(np.linalg.det(sub_a))
        
minor(a)
minor(a1)

1.0
3.0000000000000004
-3.375
-0.8859990277102598
0.019467001032006107
1.5728974209726893e-05
-2.49239946347013e-10
1.0
3.0000000000000004
0.0
-8.03305785123967
-0.4935101061624733
0.0009030145201490103
2.3183841674506325e-08


In [10]:
assert minor(a)==True
assert minor(a1)==False

1.0
3.0000000000000004
-3.375
-0.8859990277102598
0.019467001032006107
1.5728974209726893e-05
-2.49239946347013e-10


AssertionError: 

### Тест II.2

Модифицируйте алгоритм diy_lu, чтобы осуществлять выбор главного элемента в столбцах. Для контроля выбора можете использовать матрицу перестановок или массив замен.
Напишите функцию, воссоздающую изначальную матрицу из разложения. Протестируйте свой алгоритм на матрицах a и a1.

In [11]:
def diy_lu_mod(a):
    '''Perform pivoted LU factorization of the input matrix. 
    
    Parameters
    ----------
    a : np.array
        2D array representing a square matrix with float entries
       
    Returns
    -------
    P, L, U : ndarrays
        factors. Here P is a permutation matrix, L is lower triangular
        with unit diagonal elements, and U upper triangular.
    '''
    M = a.shape[0]
    
    u = a.copy()
    L = np.eye(M)
    P = np.eye(M)
    for j in range(M-1):
        lam = np.eye(M)
        
        vector = u[j:,j]
        
        maxidx=np.argmax(np.absolute(vector))
        print(maxidx)
        print(u)
        u[[j,maxidx+j],:]=u[[maxidx+j,j],:]
        L[:,[j,maxidx+j]]=L[:,[maxidx+j,j]]
        P[[j,maxidx+j],:]=P[[maxidx+j,j],:]
        
        gamma = u[j+1:, j] / u[j, j]
        
        lam[j+1:, j] = -gamma
        u = lam @ u

        lam[j+1:, j] = gamma
        L = L @ lam
    return P, L, u

def diy_lu_mod_res(a):
   P,L,U=diy_lu_mod(a)
   print(P)
   print(L)
   print(U)
   print( L@U )
   print(a)
   print(L@U - a)

print(diy_lu_mod_res(a))
print(diy_lu_mod_res(a1))

0
[[3.    3.    3.    3.    3.    3.   ]
 [3.    1.875 1.364 1.071 0.882 0.75 ]
 [3.    1.364 0.882 0.652 0.517 0.429]
 [3.    1.071 0.652 0.469 0.366 0.3  ]
 [3.    0.882 0.517 0.366 0.283 0.231]
 [3.    0.75  0.429 0.3   0.231 0.188]]
4


[[ 3.     3.     3.     3.     3.     3.   ]
 [ 0.    -1.125 -1.636 -1.929 -2.118 -2.25 ]
 [ 0.    -1.636 -2.118 -2.348 -2.483 -2.571]
 [ 0.    -1.929 -2.348 -2.531 -2.634 -2.7  ]
 [ 0.    -2.118 -2.483 -2.634 -2.717 -2.769]
 [ 0.    -2.25  -2.571 -2.7   -2.769 -2.812]]
3
[[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [ 0.000e+00 -2.250e+00 -2.571e+00 -2.700e+00 -2.769e+00 -2.812e+00]
 [ 0.000e+00  0.000e+00 -2.475e-01 -3.842e-01 -4.688e-01 -5.260e-01]
 [ 0.000e+00  2.220e-16 -1.437e-01 -2.170e-01 -2.605e-01 -2.893e-01]
 [ 0.000e+00  0.000e+00 -6.259e-02 -9.297e-02 -1.106e-01 -1.222e-01]
 [ 0.000e+00  0.000e+00 -3.506e-01 -5.786e-01 -7.330e-01 -8.438e-01]]
2
[[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [ 0.000e+00 -2.250e+00 -2.571e+00 -2.700e+00 -2.769e+00 -2.812e+00]
 [ 0.000e+00  0.000e+00 -3.506e-01 -5.786e-01 -7.330e-01 -8.438e-01]
 [ 0.000e+00  2.220e-16  0.000e+00  2.021e-02  3.998e-02  5.660e-02]
 [ 0.000e+00  0.000e+00  0.000e+00

In [12]:
P, L, U = diy_lu_mod(a)
P1, L1, U1 = diy_lu_mod(a1)

from numpy.testing import assert_allclose
assert_allclose(a, P @ L @ U, atol=1e-10)
assert_allclose(a1, P1 @ L1 @ U1, atol=1e-10 )

###BEGIN HIDDEN TESTS
np.random.seed(1234)
m = np.random.rand(N, N)
a2 = np.diag([100]*N) + m.T @ m
P2, L2, U2 = diy_lu_mod(a2)

assert_allclose(a2, P2 @ L2 @ U2)
###END HIDDEN TESTS

0
[[3.    3.    3.    3.    3.    3.   ]
 [3.    1.875 1.364 1.071 0.882 0.75 ]
 [3.    1.364 0.882 0.652 0.517 0.429]
 [3.    1.071 0.652 0.469 0.366 0.3  ]
 [3.    0.882 0.517 0.366 0.283 0.231]
 [3.    0.75  0.429 0.3   0.231 0.188]]
4
[[ 3.     3.     3.     3.     3.     3.   ]
 [ 0.    -1.125 -1.636 -1.929 -2.118 -2.25 ]
 [ 0.    -1.636 -2.118 -2.348 -2.483 -2.571]
 [ 0.    -1.929 -2.348 -2.531 -2.634 -2.7  ]
 [ 0.    -2.118 -2.483 -2.634 -2.717 -2.769]
 [ 0.    -2.25  -2.571 -2.7   -2.769 -2.812]]
3
[[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [ 0.000e+00 -2.250e+00 -2.571e+00 -2.700e+00 -2.769e+00 -2.812e+00]
 [ 0.000e+00  0.000e+00 -2.475e-01 -3.842e-01 -4.688e-01 -5.260e-01]
 [ 0.000e+00  2.220e-16 -1.437e-01 -2.170e-01 -2.605e-01 -2.893e-01]
 [ 0.000e+00  0.000e+00 -6.259e-02 -9.297e-02 -1.106e-01 -1.222e-01]
 [ 0.000e+00  0.000e+00 -3.506e-01 -5.786e-01 -7.330e-01 -8.438e-01]]
2
[[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [

1
[[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [ 0.000e+00 -2.250e+00 -2.571e+00 -2.700e+00 -2.769e+00 -2.812e+00]
 [ 0.000e+00  0.000e+00 -3.506e-01 -5.786e-01 -7.330e-01 -8.438e-01]
 [ 0.000e+00  0.000e+00  2.776e-17  2.421e-02  4.866e-02  6.961e-02]
 [ 0.000e+00  0.000e+00 -1.181e-17  0.000e+00 -5.096e-04 -1.189e-03]
 [ 0.000e+00  2.220e-16 -2.317e-17  0.000e+00 -6.462e-04 -1.516e-03]]
0
[[3.    3.    3.    3.    3.    3.   ]
 [3.    3.    1.364 1.071 0.882 0.75 ]
 [3.    1.364 0.882 0.652 0.517 0.429]
 [3.    1.071 0.652 0.469 0.366 0.3  ]
 [3.    0.882 0.517 0.366 0.283 0.231]
 [3.    0.75  0.429 0.3   0.231 0.188]]
4
[[ 3.     3.     3.     3.     3.     3.   ]
 [ 0.     0.    -1.636 -1.929 -2.118 -2.25 ]
 [ 0.    -1.636 -2.118 -2.348 -2.483 -2.571]
 [ 0.    -1.929 -2.348 -2.531 -2.634 -2.7  ]
 [ 0.    -2.118 -2.483 -2.634 -2.717 -2.769]
 [ 0.    -2.25  -2.571 -2.7   -2.769 -2.812]]
3
[[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [

AssertionError: 
Not equal to tolerance rtol=1e-07, atol=1e-10

Mismatched elements: 25 / 36 (69.4%)
Max absolute difference: 1.125
Max relative difference: 3.
 x: array([[3.      , 3.      , 3.      , 3.      , 3.      , 3.      ],
       [3.      , 1.875   , 1.363636, 1.071429, 0.882353, 0.75    ],
       [3.      , 1.363636, 0.882353, 0.652174, 0.517241, 0.428571],...
 y: array([[3.      , 3.      , 3.      , 3.      , 3.      , 3.      ],
       [3.      , 0.75    , 0.428571, 0.3     , 0.230769, 0.1875  ],
       [3.      , 1.875   , 1.363636, 1.071429, 0.882353, 0.75    ],...